# A3: A\*, IDS, and Effective Branching Factor

Anurag Kumar

In this assignment, I have implemented the Recursive Best-First Search
implementation of the A\* algorithm given in class notes of Lecture 7. The name this function is `aStarSearch`.  Also in this notebook I have included my implementaion of `iterativeDeepeningSearch` functions from the previous assignment.  I have defined a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem upto three decimal places with default precision of 0.01.

So, the functions defined are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

I have applied `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. For this I have also included my implementations of various functions, from Assignment 2:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step and also increment the `nNodes` global variable to count the number of nodes explored while searching,
  
plus the following function for the eight-tile puzzle:

  * `goalTestF_8p(state, goal)` is a boolean function which returns `True` if the current state is equal to the goal state or `False` otherwise. 
  
I have also compared their results by displaying
solution path depth, number of nodes 
generated, and the effective branching factor, and discussed the results by defining a function that prints the table as shown in the example below.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])
   
This function is defined such that it takes any number of $h$ functions in the list that is the fourth argument. But the result table is printed for $IDS$ and first three $h$ functions provided as parameter to the function.

## Heuristic Functions

For `aStarSearch` I have used the following two heuristic functions which were given in the assignment, plus I have defined the third heuristic function of my own, so for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = d$, where $d$ is the Euclidian distance that the blank is from its goal position.  It is admissible, and not constant for all states.
  
  #### Is $h3\_8p$ admissible or not?
  An **admissible heuristic** is one that never overestimates the cost of the minimum cost path from a node to the goal node. So, in case of Euclidean distance, the heuristic value is calculated using the coordinates of blank space in current state from the position of blank space in the goal state using the formula `((x1-x2)**2+(y1-y2)**2)**0.5`. Since, this value is always less than or equal to the actual cost of the path(because the tile cannot move diagonaly and the stepcost is 1 for each step). Therefore we have say that the $h3\_8p$ is admissible and it can also be verified from the results of camparing it with other heuristic functions.

## Comparison

I have applied all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

You can print a well-formatted table like the following using the $runExperiment()$.   

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course there will be one more line for `h3`.

Iterative Deepening Search and other related functions are as the one used in A2.

In [1]:
import copy
#Takes the parameters as defined in the problem definition above
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    if state == goalState:                  #to check goalState and the current state      
        return [goalState]                  #Adds the goalstate to the result list, thus initializing the solution path
    if depthLimit == 0:
        return 'cutoff'                     #Return the string 'cutoff' to signal that the depth limit was reached
    cutoffOccurred = False                  #'cutoff' flag
                                            ##'actionF' returns a list of all possible actions
    for action in actionsF(state):          #For each action in actionsF(state):
        temp=copy.copy(state)               ## this to retain the unalterd copy of state node
                                            ## A new copy of state list is passed in the takeActionF() because it performs
                                            ## the given action on that list and then returns it. Therefore childState gets
                                            ## new list not a refernce to the state list given. To remove inconsistency
        childState, stepCost = takeActionF(copy.copy(state), action)  #takes the possible action on the given state
        #in the above statement action contains [action,cost]
        
                                            # Recursive call
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':              # to check 'cutoff' flag
            cutoffOccurred = True
        elif result is not 'failure':
            #Add childState to front of partial solution path, in result, returned by depthLimitedSearch
            result.insert(0,temp)           # Since the result list already contains the goalState, just add its predecessor
            return result                   # Return the path
    if cutoffOccurred:
        return 'cutoff'                     # if depth limit exhausted
    else:
        return 'failure'                    # if all possible state exhausted i.e. goalState is not reachable from the the given
                                            # startState

In [2]:
import copy

def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global nNodes                           #this is a global variable which is being used to 
                                            #count the total number of nodes explored
    nNodes = 0                              #initialze nNodes to zero
    temp=copy.copy(startState)              ## to retain the unaltered copy of startState
    for depth in range(maxDepth):           # till maxDepth is reached, call recursively
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            #Add startState to front of solution path, in result, returned by depthLimitedSearch       
            #result.insert(0,temp) 
            # Not required to do the above steps as I have already included the startState in the depthLimitedSearch()
            return result
    return 'cutoff'

## Assumption

All 8-Puzzle functions are defined for 3*3 matrix and `0` as blank space.

`CAUTION` : The function takes large amount of time for computation if the maxDepth is more than 15. I have implemented this function on many examples to illustrate that it works fine if maxDepth is less than 15.

In [3]:
# find the blank in the state given and returns a tuple of (row,column)
def findBlank_8p(state):
    index=state.index(0)
    x=0
    y=0
    for i in range(index):
        if y<2:
            y+=1
        else:
            x+=1
            y=0
    return (x,y)

In [4]:
#to return a list of possible actions
#The actions are returned in the order 'left','right','up','down' which ever are applicable.
#The oreder is as given in the problem
def actionsF_8p(state):
    acts=[]
    x,y = findBlank_8p(state)
    if x==0:
        if y==0:
            return [('right',1),('down',1)]
        elif y==1:
            return [('left',1),('right',1),('down',1)]
        else:
            return [('left',1),('down',1)]
    elif x==1:
        if y==0:
            return [('right',1),('up',1),('down',1)]
        elif y==1:
            return [('left',1),('right',1),('up',1),('down',1)]
        else:
            return [('left',1),('up',1),('down',1)]
    else:
        if y==0:
            return [('right',1),('up',1)]
        elif y==1:
            return [('left',1),('right',1),('up',1)]
        else:
            return [('left',1),('up',1)]

In [5]:
#returns the next state after performing the action given
# I have used this function to count the number of nodes explored by incrementing nNodes for every action
def takeActionF_8p(state, action):
    global nNodes
    nNodes = nNodes+1
    #print('nNodes : ',nNodes)
    if action[0]=='up':
        x=state.index(0)
        y=state[x-3]
        state[x-3]=0
        state[x]=y
        return (state,1)
    elif action[0]=='down':
        x=state.index(0)
        y=state[x+3]
        state[x+3]=0
        state[x]=y
        return (state,1)
    elif action[0]=='left':
        x=state.index(0)
        state[x]=state[x-1]
        state[x-1]=0
        return (state,1)
    elif action[0]=='right':
        x=state.index(0)
        state[x]=state[x+1]
        state[x+1]=0
        return (state,1)

In [6]:
# This is a boolean function for comparing current state to hte goal state
def goalTestF_8p(state, goalState):
    return state==goalState

In [7]:
#prints the state of the 8-puzzle
def printState_8p(state):
    j=0
    for i in range(9):
        if state[i]!=0:
            print(state[i], end=' ')
        else:
            print(' ',end=' ')
        j+=1
        if j%3==0:
            print('',end='\n')

In [8]:
def printPath_8p(startState, goalState, path):
    print('Path from')
    printState_8p(startState)
    print('  to')
    printState_8p(goalState)
    print('is %d node long:' %(len(path)))
    for i in range(len(path)):
        #for j in range(i):
         #   print('',end=' ')
        printState_8p(path[i])
        print()

I have borrowed the A* function from the Lecture 7 notes

In [9]:
#%%writefile aStarSearch.py
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
#  Recursive Iterative Deepening form of A*, where depth is replaced by f(n)
global nNodes
class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global nNodes
    nNodes = 0
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global nNodes
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    #print(actions)
    if not actions:
        return ("failure", float('inf'))
    children = []
    #print(children)
    for action in actions:
        #print(action)
        # I have passed the copy of parentNode.state to the takeActionF becoz otherwise it alters the current state and
        # the program gets stuck in [1,2,3,4,0,5,6,7,8] state as all four actions are possible and alfer performing all the 
        # four actions on the state as a list mentioned above it reaches to the same state. And thus gets stuck in an infinite
        # loop
        (childState,stepCost) = takeActionF(copy.copy(parentNode.state), action)
        #print('childstate ',childState)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
        #print(children)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":               #        g
            result.insert(0,parentNode.state)     #       / 
            return (result, bestChild.f)          #      d
                                                  #     / \ 
if __name__ == "__main__":                        #    b   h   
                                                  #   / \   
    successors = {'a': ['b','c'],                 #  a   e  
                  'b': ['d','e'],                 #   \         
                  'c': ['f'],                     #    c   i
                  'd': ['g', 'h'],                #     \ / 
                  'f': ['i','j']}                 #      f  
    def actionsF(s):                              #       \ 
        try:                                      #        j
            ## step cost of each action is 1
            return [(succ,1) for succ in successors[s]]
        except KeyError:
            return []
    def takeActionF(s,a):
        return a
    def goalTestF(s):
        return s == goal
    def h1(s):
        return 0
    start = 'a'
    goal = 'h'
    result = aStarSearch(start,actionsF,takeActionF,goalTestF,h1)
    print('Path from a to h is', result[0], 'for a cost of', result[1])


Path from a to h is ['a', 'b', 'd', 'h'] for a cost of 3


In [10]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [11]:
def h1_8p(state,goal):
    return 0

In [12]:
def h2_8p(state,goal):
    x,y = findBlank_8p(state)
    p,q = findBlank_8p(goal)
    return abs(x-p)+abs(y-q)

In [13]:
def h3_8p(state,goal):
    x,y = findBlank_8p(state)
    p,q = findBlank_8p(goal)
    return ((x-p)**2+(y-q)**2)**.5

In [14]:
aStarSearch([1, 2, 3, 0, 4,  6, 7, 5, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

([[1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 1)

In [15]:
nNodes

7

This is the ebf() and ebf() uses cal() to do the computation

In [16]:
'''
'b' is the branching factor and 'd' is the depth.
We are passing a guess 'b' value for a particular 'd' and computing the number of nodes that may be explored
'''
def cal(b, d):
    if b!=1 and d!=1:
        return (1-b**(d+1))/(1-b)
    if d==1:
        return 1+b

$CAUTION$ Make sure that the value of depth is never greater than the number of nodes

In [17]:
'''The edf() has parameter as defined below and the precision value is 0.01 by default
    The calculation is done using bianry search and the value of ebf* is rounded upto 3 decimal places'''
def ebf(numberNodes, depth, precision=0.01):
    n = numberNodes
    d = depth
    a = n*precision
    low = 0
    high = n
    while True:
        if n==0:
            return 0.000
        if d==0:
            return 1
        b = (high+low)/2
        y_b = cal(b,d)
        if y_b<n:
            low = b
        elif y_b>n:
            high = b
        if abs(n-y_b)<a:
            return round(b,3)

#### Assumption: The hF parameter expexts a list of hueristic functions
$CAUTION$: If you want to pass just 1 hueristic function, please make sure you are passing in form of a list. 

In [18]:
'''This is the runExperiment function which compares all the four algorithms as asked in the assignment.
    This function will return error if the number of hF are less than 3 because it prints table assuming 
    3 hueristic are provided'''
def runExperiment(goalState1, goalState2, goalState3,hF):
    startState = [1,2,3,4,0,5,6,7,8]              # As given in the assignment
    ebf_data = []                                 # empty list to store data used for ebf calculation i.e nNodes and depth
    op_IDS1 = iterativeDeepeningSearch(startState, goalState1,actionsF_8p,takeActionF_8p,15)
    ebf_data.append((len(op_IDS1)-1,nNodes))
    op_IDS2 = iterativeDeepeningSearch(startState, goalState2,actionsF_8p,takeActionF_8p,15)
    ebf_data.append((len(op_IDS2)-1,nNodes))
    op_IDS3 = iterativeDeepeningSearch(startState, goalState3,actionsF_8p,takeActionF_8p,15)
    ebf_data.append((len(op_IDS3)-1,nNodes))
    op_a = [[0 for i in range(3)] for j in range(len(hF))] #'op_a' is a list of list to store the output of A* search for ith hF
    for i in range(len(hF)):
        op_a[i][0] = aStarSearch(startState, actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, goalState1),
                     lambda s: hF[i](s, goalState1))
        ebf_data.append((op_a[i][0][1],nNodes))
        op_a[i][1] = aStarSearch(startState, actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, goalState2),
                     lambda s: hF[i](s, goalState2))
        ebf_data.append((op_a[i][1][1],nNodes))
        op_a[i][2] = aStarSearch(startState, actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, goalState3),
                     lambda s: hF[i](s, goalState3))
        ebf_data.append((op_a[i][2][1],nNodes))
    ebf_values = []                                # This a list to store actual ebf values calculated using calling ebf()
    for i in range(len(ebf_data)):
        ebf_values.append(ebf(ebf_data[i][1],ebf_data[i][0]))
    '''
    print(op_IDS1)
    print()
    print(op_IDS2)
    print()
    print(op_IDS3)
    print()
    for i in range(len(op_a)):
        for j in range(len(op_a[i])):
            print(op_a[i][j])
            print()
    print()
    print('ebf data')
    print(ebf_data)
    print('ebf values')
    print(ebf_values)
    print('Table : ')'''
    print('          {}          {}            {}'.format(goalState1,goalState2,goalState3))
    print('Algorithm   Depth     Nodes     EBF               Depth     Nodes     EBF               Depth     Nodes     EBF')
    print('IDS          {}        {}         {}                {}        {}        {}             {}       {}    {}'.format(
            ebf_data[0][0],ebf_data[0][1],ebf_values[0],
            ebf_data[1][0],ebf_data[1][1],ebf_values[1],
            ebf_data[2][0],ebf_data[2][1],ebf_values[2],))
    print('A*h1         {}        {}         {}                {}        {}       {}             {}       {}    {}'.format(
            ebf_data[3][0],ebf_data[3][1],ebf_values[3],
            ebf_data[4][0],ebf_data[4][1],ebf_values[4],
            ebf_data[5][0],ebf_data[5][1],ebf_values[5],))
    print('A*h2         {}        {}         {}                {}        {}        {}             {}       {}    {}'.format(
            ebf_data[6][0],ebf_data[6][1],ebf_values[6],
            ebf_data[7][0],ebf_data[7][1],ebf_values[7],
            ebf_data[8][0],ebf_data[8][1],ebf_values[8],))
    print('A*h3         {}        {}         {}                {}        {}        {}             {}       {}    {}'.format(
            ebf_data[9][0],ebf_data[9][1],ebf_values[9],
            ebf_data[10][0],ebf_data[10][1],ebf_values[10],
            ebf_data[11][0],ebf_data[11][1],ebf_values[11],))
    

In [19]:
runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8],[1, 2, 3, 4, 5, 8, 6, 0, 7],[1, 0, 3, 4, 5, 8, 2, 6, 7],[h1_8p,h2_8p,h3_8p])

          [1, 2, 3, 4, 0, 5, 6, 7, 8]          [1, 2, 3, 4, 5, 8, 6, 0, 7]            [1, 0, 3, 4, 5, 8, 2, 6, 7]
Algorithm   Depth     Nodes     EBF               Depth     Nodes     EBF               Depth     Nodes     EBF
IDS          0        0         0.0                3        43        3.086             11       225850    2.955
A*h1         0        0         0.0                3        116       4.475             11       643246    3.264
A*h2         0        0         0.0                3        51        3.287             11       100046    2.731
A*h3         0        0         0.0                3        22        2.363             11       367417    3.093


First, some example output for the ebf function.

In [20]:
ebf(10, 3)

1.66

The smallest argument values should be a depth of 0, and 1 node.

In [21]:
ebf(1, 0)

1

In [22]:
ebf(2, 1)

1.0

In [23]:
ebf(2, 1, precision=0.000001)

1.0

In [24]:
ebf(200000, 5)

11.253

In [25]:
ebf(200000, 50)

1.235

Here is a simple example using our usual simple graph search.

In [26]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [27]:
takeActionF_simple('a', actions[0])

('b', 1)

In [28]:
goalTestF_simple('a', 'a')

True

In [29]:
h_simple('a', 'z')

1

In [30]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [31]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

In [32]:
aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h2_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

([[1, 2, 3, 4, 5, 6, 7, 0, 8],
  [1, 2, 3, 4, 0, 6, 7, 5, 8],
  [1, 2, 3, 0, 4, 6, 7, 5, 8],
  [0, 2, 3, 1, 4, 6, 7, 5, 8]],
 3)

In [33]:
aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

([[1, 2, 3, 4, 5, 6, 7, 0, 8],
  [1, 2, 3, 4, 0, 6, 7, 5, 8],
  [1, 2, 3, 0, 4, 6, 7, 5, 8],
  [0, 2, 3, 1, 4, 6, 7, 5, 8]],
 3)

In [34]:
aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h3_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

([[1, 2, 3, 4, 5, 6, 7, 0, 8],
  [1, 2, 3, 4, 0, 6, 7, 5, 8],
  [1, 2, 3, 0, 4, 6, 7, 5, 8],
  [0, 2, 3, 1, 4, 6, 7, 5, 8]],
 3)

In [35]:
iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                                 actionsF_8p, takeActionF_8p, 15)

'cutoff'

In [36]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 0, 5, 6, 7, 8]),
                     lambda s: h1_8p(s, [1, 2, 3, 4, 0, 5, 6, 7, 8]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8]], 0)

In [37]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 0, 5, 6, 7, 8]),
                     lambda s: h2_8p(s, [1, 2, 3, 4, 0, 5, 6, 7, 8]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8]], 0)

In [38]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 0, 5, 6, 7, 8]),
                     lambda s: h3_8p(s, [1, 2, 3, 4, 0, 5, 6, 7, 8]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8]], 0)

In [39]:
iterativeDeepeningSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],[1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p,takeActionF_8p,10)

[[1, 2, 3, 4, 0, 5, 6, 7, 8]]

In [40]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]),
                     lambda s: h1_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 5, 8, 6, 7, 0],
  [1, 2, 3, 4, 5, 8, 6, 0, 7]],
 3)

In [41]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]),
                     lambda s: h2_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 5, 8, 6, 7, 0],
  [1, 2, 3, 4, 5, 8, 6, 0, 7]],
 3)

In [42]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]),
                     lambda s: h3_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 5, 8, 6, 7, 0],
  [1, 2, 3, 4, 5, 8, 6, 0, 7]],
 3)

In [43]:
iterativeDeepeningSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],[1, 2, 3, 4, 5, 8, 6, 0, 7],actionsF_8p,takeActionF_8p,10)

[[1, 2, 3, 4, 0, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 0, 6, 7, 8],
 [1, 2, 3, 4, 5, 8, 6, 7, 0],
 [1, 2, 3, 4, 5, 8, 6, 0, 7]]

In [44]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]),
                     lambda s: h1_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 0, 3, 4, 2, 5, 6, 7, 8],
  [0, 1, 3, 4, 2, 5, 6, 7, 8],
  [4, 1, 3, 0, 2, 5, 6, 7, 8],
  [4, 1, 3, 2, 0, 5, 6, 7, 8],
  [4, 1, 3, 2, 5, 0, 6, 7, 8],
  [4, 1, 3, 2, 5, 8, 6, 7, 0],
  [4, 1, 3, 2, 5, 8, 6, 0, 7],
  [4, 1, 3, 2, 5, 8, 0, 6, 7],
  [4, 1, 3, 0, 5, 8, 2, 6, 7],
  [0, 1, 3, 4, 5, 8, 2, 6, 7],
  [1, 0, 3, 4, 5, 8, 2, 6, 7]],
 11)

In [45]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]),
                     lambda s: h2_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 0, 3, 4, 2, 5, 6, 7, 8],
  [0, 1, 3, 4, 2, 5, 6, 7, 8],
  [4, 1, 3, 0, 2, 5, 6, 7, 8],
  [4, 1, 3, 2, 0, 5, 6, 7, 8],
  [4, 1, 3, 2, 5, 0, 6, 7, 8],
  [4, 1, 3, 2, 5, 8, 6, 7, 0],
  [4, 1, 3, 2, 5, 8, 6, 0, 7],
  [4, 1, 3, 2, 5, 8, 0, 6, 7],
  [4, 1, 3, 0, 5, 8, 2, 6, 7],
  [0, 1, 3, 4, 5, 8, 2, 6, 7],
  [1, 0, 3, 4, 5, 8, 2, 6, 7]],
 11)

In [46]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]),
                     lambda s: h3_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 0, 3, 4, 2, 5, 6, 7, 8],
  [0, 1, 3, 4, 2, 5, 6, 7, 8],
  [4, 1, 3, 0, 2, 5, 6, 7, 8],
  [4, 1, 3, 2, 0, 5, 6, 7, 8],
  [4, 1, 3, 2, 5, 0, 6, 7, 8],
  [4, 1, 3, 2, 5, 8, 6, 7, 0],
  [4, 1, 3, 2, 5, 8, 6, 0, 7],
  [4, 1, 3, 2, 5, 8, 0, 6, 7],
  [4, 1, 3, 0, 5, 8, 2, 6, 7],
  [0, 1, 3, 4, 5, 8, 2, 6, 7],
  [1, 0, 3, 4, 5, 8, 2, 6, 7]],
 11)

In [47]:
iterativeDeepeningSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],[1, 0, 3, 4, 5, 8, 2, 6, 7],actionsF_8p,takeActionF_8p,10)

'cutoff'

In [48]:
iterativeDeepeningSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],[1, 0, 3, 4, 5, 8, 2, 6, 7],actionsF_8p,takeActionF_8p,15)

[[1, 2, 3, 4, 0, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [0, 1, 3, 4, 2, 5, 6, 7, 8],
 [4, 1, 3, 0, 2, 5, 6, 7, 8],
 [4, 1, 3, 2, 0, 5, 6, 7, 8],
 [4, 1, 3, 2, 5, 0, 6, 7, 8],
 [4, 1, 3, 2, 5, 8, 6, 7, 0],
 [4, 1, 3, 2, 5, 8, 6, 0, 7],
 [4, 1, 3, 2, 5, 8, 0, 6, 7],
 [4, 1, 3, 0, 5, 8, 2, 6, 7],
 [0, 1, 3, 4, 5, 8, 2, 6, 7],
 [1, 0, 3, 4, 5, 8, 2, 6, 7]]

In [49]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            